In [93]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import json
from joblib import dump, load

In [94]:
# load the dataset into a Pandas DataFrame
df = pd.read_csv('./bio_features.csv', index_col=0)

In [95]:
df.shape

In [96]:
df.head()

In [97]:
# drop the columns that are not needed
df = df.drop(["s_id", "t_id"], axis=1)

df.shape

In [98]:
# create the correlation matrix
corr_matrix = df.corr()

# create a heatmap using seaborn
sns.heatmap(corr_matrix, cmap='coolwarm', annot=False)

In [99]:
# extract the features and target variable
X = df.drop(["surprise", "anxiety", "boredom", "calmness", "comfort"], axis=1)
y = df[["surprise", "anxiety", "boredom", "calmness", "comfort"]]

In [100]:
X.shape

In [101]:
y.shape

In [102]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [103]:
# initialize the StandardScaler object
scaler = StandardScaler()

# fit and transform the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [104]:
pd.DataFrame(X_train_scaled, columns=X_train.columns).head()

In [105]:
print("length of X_train_scaled:", len(X_train_scaled))
print("length of y_train:", len(y_train))
print("length of X_test_scaled:", len(X_test_scaled))
print("length of y_test:", len(y_test))

In [106]:
# Define the parameter grid for Grid Search
param_grid = {
    'n_estimators': [16, 100, 400]
}

# Perform Grid Search to find the best value of n_estimators
grid_search = GridSearchCV(estimator=RandomForestRegressor(), param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Print the best value of n_estimators
print("Best value of n_estimators:", grid_search.best_params_['n_estimators'])

# Save the best parameters to a file
best_params = grid_search.best_params_
# with open('./saved_models/best_params_rf.json', 'w') as f:
#     json.dump(best_params, f)

# Evaluate the model with the best value of n_estimators on the testing set
y_pred = grid_search.predict(X_test)

In [107]:
rf = RandomForestRegressor(n_estimators=best_params[0], random_state=42)
rf.fit(X_train_scaled, y_train)

In [ ]:
# Save the model to a file
dump(rf, './saved_models/random_forest.joblib')

In [ ]:
# Convert the scaled training data to a DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)

# Extract the feature names from the DataFrame columns
feature_names = X_train_scaled_df.columns.tolist()

# Get feature importances from the trained model
importances = rf.feature_importances_

# summarize feature importance
for i,v in enumerate(importances):
 print('Feature: %0d, Score: %.5f' % (i,v))

# plot feature importance
plt.bar([x for x in range(len(importances))], importances)
plt.show()

In [ ]:
# Extract the feature names from the DataFrame columns
feature_names = X_train_scaled_df.columns.tolist()

# Sort the feature importances in descending order
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(X_train.shape[1]):
    print("%d. %s (%f)" % (f + 1, feature_names[indices[f]], importances[indices[f]]))

# Plot the feature importances
plt.figure()
plt.title("Feature importances")
plt.bar(range(X_train.shape[1]), importances[indices], color="b", align="center")
plt.xticks(range(X_train.shape[1]), [feature_names[i] for i in indices], rotation=45, ha='right')
plt.xlim([-1, X_train.shape[1]])
plt.show()

In [ ]:
print("Shape of y_pred:", y_pred.shape)
print("Shape of y_test:", y_test.shape)

In [ ]:
def compute_ccc(y_test, y_pred):
    """
    Compute the Concordance correlation coefficient between y_true and y_pred.
    """
    mean_test = np.mean(y_test)
    mean_pred = np.mean(y_pred)
    var_test = np.var(y_test)
    var_pred = np.var(y_pred)
    covar = np.cov(y_test, y_pred)[0, 1]
    rho_c = 2 * covar / (var_test + var_pred + (mean_test - mean_pred) ** 2)
    return rho_c

In [ ]:
# evaluate the model using root mean squared error
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
ccc = compute_ccc(y_test, y_pred)

print("Mean squared error:", mse)
print("Root Mean squared error:", rmse)
print("Mean Absolute error:", mae)
# print("Pearson correlation:", pearsonr(y_test, y_pred))
print("Concordance correlation coefficients (CCC):")
print(ccc)
